In [ ]:
import os
import pickle
import pandas as pd

In [ ]:
home_string = ''
os.chdir(home_string)

In [ ]:
%load_ext autoreload
%autoreload 2
from main.training.train_routines import train_boosting_model, train_linear_models, train_fix_onetonets
from main.training.sampling import sample_data_sim

In [ ]:
want_data = pd.read_csv('data/application/data_selection/data_reg_reg.csv')
want_data = want_data.drop(columns='POBP')
# Set up to scale
target_cols = ['PINCP', 'JWMNP']
numeric_cols = ['SCHL', 'WKHP', 'AGEP', 'POVPIP']
other_cols = want_data.drop(columns = target_cols + numeric_cols).columns

feature_df = want_data.drop(columns=target_cols)
label_df = want_data.loc[:,target_cols]

data_dict, scaler_dict, sens_idx = sample_data_sim(feature_df, label_df, 42, scale_y=True,
                                         other_cols=other_cols, numeric_cols=numeric_cols, 
                                         )

In [ ]:
d = data_dict['X_train'].shape[1]
hyper_params_oneto = {
 'learning_rate' : [0.05,0.10,0.15],
 'lambda_' : [10e-5, 10e-4, 10e-2, 0.5],
 'gamma' : [1,4,10],
 'r' : [2^2*d, 2^3*d, 2^4*d]
}

hyper_params_boost = {
 'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
 'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
 'min_child_weight' : [ 1, 3, 5, 7 ],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [ ]:
seeds = [102, 435, 860, 270, 106,  71, 700,
         20, 614, 121, 466, 214, 330,
         458,  87, 372,  99, 871, 663, 130]

In [ ]:
for seed_ in seeds[-5:]:
    data_dict, scaler_dict, sens_feature_idx = sample_data_sim(feature_df, label_df, seed_, scale_y=True)

    y_test_orig = scaler_dict['scaler_y'].inverse_transform(data_dict['y_test'])
    oneto_results = train_fix_onetonets(data_dict, hyper_params_oneto, sens_feature_idx)
    oneto_results = scaler_dict['scaler_y'].inverse_transform(oneto_results)


    prediction_dict = {   
    'orig_data': y_test_orig, 
    'all_test_data': data_dict['X_test'],
    'data_out_sens': sens_feature_idx, 
    'data_oneto': oneto_results
    }

    with open(f'data/application/results/results_seed_oneto_{seed_}.pkl', 'wb') as con_:
        pickle.dump(prediction_dict, con_)

    boosting_results = train_boosting_model(data_dict, hyper_params_boost, 10)
    lasso_results = train_linear_models(data_dict, 10)

    boosting_ = scaler_dict['scaler_y'].inverse_transform(boosting_results)
    lasso_ = scaler_dict['scaler_y'].inverse_transform(lasso_results)
    y_test_orig = scaler_dict['scaler_y'].inverse_transform(data_dict['y_test'])

    prediction_dict = {   
    'orig_data': y_test_orig, 
    'all_test_data': data_dict['X_test'],
    'data_out_sens': sens_feature_idx, 
    'preds_boosting_': boosting_, 
    'preds_lasso_': lasso_, 
    }

    with open(f'data/application/results/results_seed_{seed_}.pkl', 'wb') as con_:
        pickle.dump(prediction_dict, con_)
